In [1]:
from pynsee.localdata import *
from pynsee.geodata import *
from pynsee.metadata import *
from shapely import wkt

import pandas as pd
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import descartes
import geopandas as gpd

# get a list all data available : datasets and variables
metadata = get_local_metadata()

# geographic metadata
nivgeo = get_nivgeo_list()

!!! This function renders only package's internal data, it might not be the most up-to-date
Have a look at api.insee.fr !!!
!!! This function renders only package's internal data,
it might not be the most up-to-date
Have a look at api.insee.fr !!!


In [2]:
# get departement map
dep = get_geodata('ADMINEXPRESS-COG-CARTO.LATEST:departement')
dep = dep[["insee_dep", "geometry"]]
# remove overseas departments
dep = dep[~dep['insee_dep'].isin(['971', '972', '973', '974', '976'])]
dep = dep.rename(columns={'insee_dep': 'CODEGEO'})

Previously saved data used
/home/jovyan/.cache/pynsee/pynsee/a28d68dc88fcd55fe62626ea25a1760a.csv
Set update=True to get the most up-to-date data


In [11]:
data = get_local_data(dataset_version = 'GEO2020FLORES2017',
                     variables =  'NA17',
                     nivgeo = 'DEP')
data = data.rename(columns={'NA17': 'A17'})

Getting data: 100%|██████████| 102/102 [00:00<00:00, 451285.87it/s]


In [13]:
df = data[data["UNIT"] == "NBSALET"].reset_index(drop=True)
df = df[df["A17"] != "ENS"].reset_index(drop=True)

In [14]:
# compute share of activity in total employment by department
df['employment_share'] = 100 * df["OBS_VALUE"] / df['OBS_VALUE'].groupby(df['CODEGEO']).transform('sum')
df['employment_share']

0        1.281165
1        2.515339
2        0.000000
3        0.180870
4        0.030145
          ...    
1712     2.297605
1713     0.937722
1714     8.525423
1715    36.085478
1716     4.018007
Name: employment_share, Length: 1717, dtype: float64

In [15]:
# merge values and geographic limits
datamap = dep.merge(df, how = 'left', on = 'CODEGEO')
datamap = gpd.GeoDataFrame(datamap).set_crs("EPSG:4326")

In [16]:
activity = get_activity_list("A17")
datamap = datamap.merge(activity, how='left', on='A17')
datamap

,CODEGEO,geometry,NIVGEO,UNIT,UNIT_label,A17,OBS_VALUE,NA17_label,DATASET_VERSION,DATASET_NAME,DATA_DATE,GEO_DATE,CODEGEO_label,employment_share,A10,TITLE_A17_EN,TITLE_A17_FR
0,01,"MULTIPOLYGON (((4.79539 46.22007, 4.79676 46.2...",DEP,NBSALET,Postes salariés dans les établissements,AZ,1577.0,"Agriculture, sylviculture et pêche",GEO2020FLORES2017,Fichier localisé des rémunérations et de l'emp...,2017,2020,Ain,0.845141,AZ,"Agriculture, forestry and fishing","Agriculture, sylviculture et pêche"
1,01,"MULTIPOLYGON (((4.79539 46.22007, 4.79676 46.2...",DEP,NBSALET,Postes salariés dans les établissements,C1,4964.0,"Fabrication de denrées alimentaires, de boisso...",GEO2020FLORES2017,Fichier localisé des rémunérations et de l'emp...,2017,2020,Ain,2.660293,BE,"Manufacture of food products, beverages and to...","Fabrication de denrées alimentaires, de boisso..."
2,01,"MULTIPOLYGON (((4.79539 46.22007, 4.79676 46.2...",DEP,NBSALET,Postes salariés dans les établissements,C2,0.0,Cokéfaction et raffinage,GEO2020FLORES2017,Fichier localisé des rémunérations et de l'emp...,2017,2020,Ain,0.000000,BE,Manufacture of coke and refined petroleum prod...,Cokéfaction et raffinage
3,01,"MULTIPOLYGON (((4.79539 46.22007, 4.79676 46.2...",DEP,NBSALET,Postes salariés dans les établissements,C3,7195.0,"Fabrication d'équipements électriques, électro...",GEO2020FLORES2017,Fichier localisé des rémunérations et de l'emp...,2017,2020,Ain,3.855924,BE,"Manufacture of electrical, computer and electr...","Fabrication d'équipements électriques, électro..."
4,01,"MULTIPOLYGON (((4.79539 46.22007, 4.79676 46.2...",DEP,NBSALET,Postes salariés dans les établissements,C4,4242.0,Fabrication de matériels de transport,GEO2020FLORES2017,Fichier localisé des rémunérations et de l'emp...,2017,2020,Ain,2.273361,BE,Manufacture of transport equipment,Fabrication de matériels de transport
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1644,95,"MULTIPOLYGON (((2.43548 49.13405, 2.43755 49.1...",DEP,NBSALET,Postes salariés dans les établissements,KZ,7234.0,Activités financières et d'assurance,GEO2020FLORES2017,Fichier localisé des rémunérations et de l'emp...,2017,2020,Val-d'Oise,1.865601,KZ,Financial and insurance activities,Activités financières et d'assurance
1645,95,"MULTIPOLYGON (((2.43548 49.13405, 2.43755 49.1...",DEP,NBSALET,Postes salariés dans les établissements,LZ,3334.0,Activités immobilières,GEO2020FLORES2017,Fichier localisé des rémunérations et de l'emp...,2017,2020,Val-d'Oise,0.859817,LZ,Real estate activities,Activités immobilières
1646,95,"MULTIPOLYGON (((2.43548 49.13405, 2.43755 49.1...",DEP,NBSALET,Postes salariés dans les établissements,MN,48690.0,Activités scientifiques et techniques ; servic...,GEO2020FLORES2017,Fichier localisé des rémunérations et de l'emp...,2017,2020,Val-d'Oise,12.556833,MN,"Professional, scientific, technical, administr...",Activités scientifiques et techniques ; servic...
1647,95,"MULTIPOLYGON (((2.43548 49.13405, 2.43755 49.1...",DEP,NBSALET,Postes salariés dans les établissements,OQ,124402.0,"Administration publique, enseignement, santé h...",GEO2020FLORES2017,Fichier localisé des rémunérations et de l'emp...,2017,2020,Val-d'Oise,32.082464,OQ,"Public administration and defence, education, ...","Administration publique, enseignement, santé h..."


In [8]:
def get_center(x):
    return x.centroid.wkt
datamap["center"] = datamap['geometry'].apply(lambda x: get_center(x))


In [18]:
import colorsys
N = 5
HSV_tuples = [(x*1.0/N, 0.5, 0.5) for x in range(N)]
RGB_tuples = map(lambda x: colorsys.hsv_to_rgb(*x), HSV_tuples)
